<a href="https://colab.research.google.com/github/Sen-Ma/PUBG-Kaggle/blob/master/Effective_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gc
import time 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

import random 
random.seed(42)

import warnings 
warnings.filterwarnings("ignore", category = DeprecationWarning)

def reload():
  gc.collect()
  df = pd.read_csv('/content/gdrive/My Drive/pubg/train_V2.csv')
  invalid_match_id = df[df['winPlacePerc'].isna()]['matchId'].values
  df = df[-df['matchId'].isin(invalid_match_ids)]
  return df

# train = pd.read_csv("/content/gdrive/My Drive/pubg/train_V2.csv")

In [0]:
df = reload()
cols_to_drop = ['Id', 'groupId', 'matchId', 'matchType']
cols_to_fit = [col for col in df.columns if col bot in cols_to_drop]
corr = df[clos_to_fit].corr()

plt.figure(figsize = (9,7))
sns.headmap(
    corr,
    xticklabels = corr.columns.values,
    yticklabels = corr.columns.values,
    linecolor = 'white', 
    linewidths = 0.1,
    cmap = "RdBu"
)

plt.show()

agg = df.groupby9['groupId']).size().to_frame('players_in_team')
df = df.merge(agg,how='left', on = ['groupId'])
df['headshotKills_over_kills'] = df['headshotKills'] / df['kills']
df['headshotKills_over_kills'].fillna(0, inplace =True)
df['killPlace_over_maxPlace'] = df['killPlace'] / df['maxPlace']